In [31]:
import numpy as np
import tensorflow as tf
import os

from triplet_dataset_loader import *
from TL_class import SiameseTripletModel
from TL_siamese_network import generate_siamese_triplet_network
from top_accuracy_functions import *

In [36]:
database_path = "../databases/my-fruit-recognition"
split_ratio = (0.8, 0.1999999999999999999, 0.0000000000000000001)
image_size = (100,100)
batch_size = 32

auto = tf.data.AUTOTUNE

learning_rate = 0.0001
steps_per_epoch = 50
validation_steps = 10
epochs = 3

train_dataset, val_dataset, test_dataset = create_dataset(database_path, split_ratio, image_size, batch_size)
triplet_siamese_network = generate_siamese_triplet_network(image_size)
triplet_siamese_model = SiameseTripletModel(triplet_siamese_network)
triplet_siamese_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
triplet_siamese_model.summary()

Model: "siamese_triplet_model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ functional_7 (Functional)       │ ?                      │    40,565,632 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,565,632 (154.75 MB)

 Trainable params: 25,907,712 (98.83 MB)

 Non-trainable params: 14,657,920 (55.92 MB)

In [39]:
len(test_dataset)

TypeError: The dataset length is unknown.

In [38]:
history = triplet_siamese_model.fit(
    train_dataset,
    validation_data=val_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=epochs
)

Epoch 1/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.2424 - loss: 0.5746 - val_accuracy: 0.2188 - val_loss: 0.4017
Epoch 2/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.2156 - loss: 0.4241 - val_accuracy: 0.2281 - val_loss: 0.3915
Epoch 3/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.3128 - loss: 0.3513 - val_accuracy: 0.2562 - val_loss: 0.3938


# -------------------------------------------------------------------------------------------

In [40]:
embedding_model = triplet_siamese_network.get_layer("Embedding")

In [41]:
base_test_folder = "../databases/my-fruit-recognition"
reference_images_dict = {}

for root, dirs, files in os.walk(base_test_folder):
    files = [f for f in files if f != ".DS_Store"]
    if files:
        label_name = os.path.basename(root)
        files_sorted = sorted(files)
        reference_image_path = os.path.join(root, files_sorted[0])
        if label_name not in reference_images_dict:
            reference_images_dict[label_name] = []
        reference_images_dict[label_name].append(reference_image_path)
        reference_image_paths.append(reference_image_path)

for class_name, image_paths in reference_images_dict.items():
    print(f"Class: {class_name}")
    for path in image_paths:
        print(path)

Class: Apple E
../databases/my-fruit-recognition/Apple/Apple E/Apple E001.png
Class: Apple B
../databases/my-fruit-recognition/Apple/Apple B/102red applee00901102.png
Class: Apple C
../databases/my-fruit-recognition/Apple/Apple C/Apple Ce00151.png
Class: Apple D
../databases/my-fruit-recognition/Apple/Apple D/Apple De00001.png
Class: Apple A
../databases/my-fruit-recognition/Apple/Apple A/Apple 1.png
Class: Apple F
../databases/my-fruit-recognition/Apple/Apple F/Apple F _0_1.png
Class: Pitaya
../databases/my-fruit-recognition/Pitaya/Pitaya001.png
Class: Pear
../databases/my-fruit-recognition/Pear/Pear 1.png
Class: Pomegranate
../databases/my-fruit-recognition/Pomegranate/Pomegranet001.png
Class: Tomatoes
../databases/my-fruit-recognition/Tomatoes/Tamotoes001.png
Class: kiwi A
../databases/my-fruit-recognition/Kiwi/kiwi A/Kiwi A001.png
Class: Kiwi C
../databases/my-fruit-recognition/Kiwi/Kiwi C/Kiwi C001.png
Class: Kiwi B
../databases/my-fruit-recognition/Kiwi/Kiwi B/Kiwi B001.png
Class

In [42]:
mapF = MapFunction(image_size)
reference_embeddings_dict = {}

for class_name, class_images in reference_images_dict.items():
    class_images_resized = [mapF.decode_and_resize(image_path) for image_path in class_images]
    class_images_tensor = tf.convert_to_tensor(class_images_resized)
    class_embeddings = embedding_model.predict(class_images_tensor)
    reference_embeddings_dict[class_name] = class_embeddings

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [43]:
predicted_embeddings = []

for batch in test_dataset:
    # Extract the images from the batch
    images = batch[0]  # Assuming the images are the first element of each batch

    # Predict embeddings for the images in the batch
    batch_embeddings = embedding_model.predict(images)

    # Append the predicted embeddings to the list
    predicted_embeddings.append(batch_embeddings)

# Concatenate the predicted embeddings from all batches into a single tensor
predicted_embeddings = np.concatenate(predicted_embeddings, axis=0)


# way too long

2024-05-27 09:33:06.315674: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: IndexError: Cannot choose from an empty sequence
Traceback (most recent call last):

  File "/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/jcbob/Documents/repositories/github/projekt_zestpolowy_github/siamese-network-image-classification/triplet_dataset_loader.py", line 87

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} IndexError: Cannot choose from an empty sequence
Traceback (most recent call last):

  File "/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/jcbob/Documents/repositories/github/projekt_zestpolowy_github/siamese-network-image-classifi